<a href="https://colab.research.google.com/github/Ahmedsayed31/Seq2seq-Eng_to_Ara/blob/main/Seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
from pathlib import Path
import pandas as pd

In [3]:
data_path = '/content/ara.txt'

In [4]:
with open(data_path, 'r',encoding='utf-8') as f:
    text = f.read()
    text=text.split('\n')

In [5]:
for i in range(5):
  print(text[i])

Hi.	مرحبًا.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #629296 (Samer)
Run!	اركض!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1245450 (saeb)
Duck!	اخفض رأسك!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036391 (KeEichi)
Duck!	اخفضي رأسك!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036392 (KeEichi)
Duck!	اخفضوا رؤوسكم!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036393 (KeEichi)


In [6]:
# splite sentences to Arabic and English
input = []
target = []
_=[]
for i ,e in enumerate(text[:-1]):
    a,b,c =e.split('\t')
    input.append(a)
    target.append(b)
    _ .append(c)


In [7]:
df = pd.DataFrame(list(zip(input, target)),columns=['input','target'])

In [8]:
df.sample(10)

,input,target
6156,There are no clean plates.,ليس هناك أطباق نظيفة.
5699,This is how we cook rice.,هكذا نطهي الأرز
8947,I want to ask you one last favor.,أريد أن أطلب منك خدمة أخيرة.
10475,Tom actually did what he said he'd do.,قام توم بعمل ما قد قال انه سيفعله حقا
6278,You will be busy tomorrow.,ستكون مشغولاً في الغد.
11335,I just want a straight answer. Nothing more.,أريد فقط إجابة صريحة، لا أكثر.
4942,He sometimes watches TV.,يشاهد التلفاز أحياناً.
1629,I am your father.,أنا أبوك.
11284,What's the best way to resolve the problem?,ما افضل طريقه لحل المشكله
225,All aboard!,ليركب الجميع.


In [9]:
print(df.input.shape)
print(df.target.shape)

(12523,)
(12523,)


In [10]:
# Prepare target sequence for decoder input by adding start and end tokens
df['target'] = df['target'].apply(lambda x:f'<start> {x} <end>')

In [11]:
English = df.input.values
Arabic = df.target.values

In [12]:
Arabic[:5]

array(['<start> مرحبًا. <end>', '<start> اركض! <end>',
       '<start> اخفض رأسك! <end>', '<start> اخفضي رأسك! <end>',
       '<start> اخفضوا رؤوسكم! <end>'], dtype=object)

## Train and Test Data

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train , x_test , y_train , y_test = train_test_split(English,Arabic,test_size=0.2,random_state=42)

## Tokenization

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
def tokenize(x):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(x)
  return tokenizer

In [17]:
encoder_tokeniz = tokenize(English)
decoder_tokeniz = tokenize(Arabic)

encoder_vocab = encoder_tokeniz.word_index
decoder_vocab = decoder_tokeniz.word_index

encoder_vocab_size = len(encoder_vocab)+1
decoder_vocab_size = len(decoder_vocab)+1

print("encoder Voca size",encoder_vocab_size)
print("decoder Voca size",decoder_vocab_size)

encoder Voca size 4422
decoder Voca size 13597


In [18]:
[max(len(i.split()) for i in Arabic)]

[38]

In [19]:
def prepare_sequences(x):
    tokenizer1 = Tokenizer()
    tokenizer1.fit_on_texts(x)
    length = max([len(i.split()) for i in x])  # استخدم split() للحصول على طول الكلمات
    x_seq = tokenizer1.texts_to_sequences(x)
    x_padded = pad_sequences(x_seq, maxlen=length, padding='post')
    return x_padded


x_train_seq = prepare_sequences(x_train)
x_test_seq = prepare_sequences(x_test)
y_train_seq = prepare_sequences(y_train)
y_test_seq = prepare_sequences(y_test)

In [20]:
x_train_seq.shape

(10018, 34)

## LSTM Encoder Decoder

In [21]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [22]:
# Encoder
encoder_input_layer = Input(shape=(None,))
encoder_embedding_layer = Embedding(encoder_vocab_size, 256)(encoder_input_layer)
encoder_lstm_layer = LSTM(512, return_state=True)
output_encoder , hidden_state, state_c = encoder_lstm_layer(encoder_embedding_layer)
encoder_states = [hidden_state, state_c]

# Decoder
decoder_input_layer = Input(shape=(None,))
decoder_embedding_layer = Embedding(decoder_vocab_size, 256)(decoder_input_layer)
decoder_lstm_layer = LSTM(512,return_sequences=True)(decoder_embedding_layer,initial_state=encoder_states)
decoder_output_layer = Dense(decoder_vocab_size,activation='softmax')(decoder_lstm_layer)

In [23]:
# Encoder
encoder_input_layer = Input(shape=(None,))
encoder_embedding_layer = Embedding(encoder_vocab_size, 256)(encoder_input_layer)

# First LSTM layer
encoder_lstm_1 = LSTM(256, return_sequences=True)(encoder_embedding_layer)
# Second LSTM layer (additional hidden layer)
encoder_lstm_2 = LSTM(256, return_state=True)(encoder_lstm_1)
output_encoder, hidden_state, state_c = encoder_lstm_2
encoder_states = [hidden_state, state_c]

# Decoder
decoder_input_layer = Input(shape=(None,))
decoder_embedding_layer = Embedding(decoder_vocab_size, 256)(decoder_input_layer)

# First LSTM layer in decoder
decoder_lstm_1 = LSTM(256, return_sequences=True)(decoder_embedding_layer, initial_state=encoder_states)
# Second LSTM layer (additional hidden layer)
decoder_lstm_2 = LSTM(256, return_sequences=True)(decoder_lstm_1)
decoder_output_layer = Dense(decoder_vocab_size, activation='softmax')(decoder_lstm_2)

# Define the model
decoder_model = Model([encoder_input_layer, decoder_input_layer], decoder_output_layer)
decoder_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 256) │  1,132,032 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, None, 256) │    525,312 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 256) │  3,480,832 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 256),     │    525,312 │ lstm_2[0][0]      │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, None, 256) │    525,312 │ embedding_3[0][0… │
│                     │                   │            │ lstm_3[0][1],     │
│                     │                   │            │ lstm_3[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, None, 256) │    525,312 │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None,      │  3,494,429 │ lstm_5[0][0]      │
│                     │ 13597)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,208,541 (38.94 MB)

 Trainable params: 10,208,541 (38.94 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
decoder_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
train=[x_train_seq,y_train_seq[:,:-1]]
test=[x_test_seq,y_test_seq[:,:-1]]

In [26]:
decoder_model.fit(train,y_train_seq[:,1:],epochs=25,batch_size=32,validation_data=(test,y_test_seq[:,1:]))

Epoch 1/25
314/314 ━━━━━━━━━━━━━━━━━━━━ 22s 52ms/step - accuracy: 0.8406 - loss: 2.0933 - val_accuracy: 0.7634 - val_loss: 2.0411
Epoch 2/25
314/314 ━━━━━━━━━━━━━━━━━━━━ 19s 52ms/step - accuracy: 0.8833 - loss: 0.9768 - val_accuracy: 0.7607 - val_loss: 2.0109
Epoch 3/25
314/314 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - accuracy: 0.8837 - loss: 0.9636 - val_accuracy: 0.7614 - val_loss: 2.0105
Epoch 4/25
314/314 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - accuracy: 0.8829 - loss: 0.9616 - val_accuracy: 0.7608 - val_loss: 2.0193
Epoch 5/25
314/314 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.8842 - loss: 0.9461 - val_accuracy: 0.7626 - val_loss: 2.0230
Epoch 6/25
314/314 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.8834 - loss: 0.9493 - val_accuracy: 0.7561 - val_loss: 2.0504
Epoch 7/25
314/314 ━━━━━━━━━━━━━━━━━━━━ 21s 51ms/step - accuracy: 0.8830 - loss: 0.9506 - val_accuracy: 0.7636 - val_loss: 2.0413
Epoch 8/25
314/314 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.8833 - loss: 0.9382 - 

In [35]:
decoder_model.evaluate([x_test_seq,y_test_seq[:,:-1]],y_test_seq[:,1:])

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7571 - loss: 2.1800


[2.231189250946045, 0.7539809346199036]